<header style="background-color: rgb(0, 62, 92); color: white; margin-top: 20px; padding:28px; ">
  <img src="Xlogo.png" alt="Transposition of a vector" title="Vector transposition" width="115" style="float: left;">
  <p style=" text-align: center; font-size: 32px;">   
  <strong> MONTE CARLO CDO TRANCHE PRICING  </strong></p>
  <p style=" text-align: center; font-size: 25px;"><strong> </strong></p>
  <p style=" text-align: center; font-size: 20px;">YELPOUGDOU INOUSSA   </p>
</header>

Dans ce notebook, nous allons essayer de pricer, avec plusieurs différentes méthodes en **monte carlo**, des tranches de CDO synthétiques. L'idée centrale consiste a comparer plusieurs différentes méthodes entre elles pour juger de la qualité, des forces et faiblesses de chacune d'entre elles. Parmi les différentes méthodes, nous aurons entre autres :

- La méthode de la copule gaussienne

- La méthode de la copule de Student

- La méthode de la copule de la fonction de répartition de la normale inverse gaussienne


Mais avant de commencer a implémenter les méthodes citées plus haut, il est important d'importer les librairies nécessaires et indispensables au calcul scientifique en Python ( Scipy )  et celui indispensable au calcul optimisé des tableaux ( Numpy )

In [1]:
import math
import numpy as np
from scipy.stats import norm
from scipy.stats import multivariate_normal
from scipy.stats import t
from scipy.stats import norm, norminvgauss
from scipy.integrate import quad
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, module="numpy.core.fromnumeric")
warnings.filterwarnings("ignore", category=RuntimeWarning, module="numpy.core._methods")

**Méthode de la copule Gaussienne a un facteur et Approximation LHP**

La méthode de la copule gaussienne est un des modèles mathématiques les plus utilisés en finance pour déterminer le prix des tranches de CDO synthétiques. Le principe est le suivant :


Le modèle LHP est basé sur une copule gaussienne à un facteur de défauts corrélés. Supposons que le portefeuille d'actifs de référence se compose de $m$ instruments financiers et que le rendement de l'actif jusqu'au temps $t$ du $i$-ème émetteur du portefeuille, $A_i(t)$, ait la forme :
\begin{equation}
A_i(t) = a_i M(t) + \sqrt{1 - a_i^2} X_i(t), \qquad
\end{equation}
où $M(t)$, $X_i(t)$, $i = 1, \dots, m$, sont des variables aléatoires indépendantes suivant une distribution normale standard. Alors, conditionnellement au facteur de marché commun $M(t)$, les rendements des actifs des différents émetteurs sont indépendants. On remarque que, en raison de la stabilité des distributions normales sous convolution, le rendement de l'actif $A_i(t)$ suit également une distribution normale standard.

Dans ce modèle de copule, la variable $A_i(t)$ est transformée en temps de défaut $t_i$ du $i$-ème émetteur en utilisant une transformation percentile-à-percentile, c'est-à-dire
\begin{equation}
\mathbb{P}[\tau_i \leq t] = \mathbb{P}[A_i(t) \leq C_i(t)]. \qquad
\end{equation}
Nous notons la probabilité pour l'émetteur $i$ de faire défaut avant le temps $t$ par
\begin{equation}
q_i(t) = \mathbb{P} [\tau_i \leq t]. \qquad
\end{equation}
Les probabilités neutres au risque sont déduites des prix du marché observables des instruments de défaut de crédit (par exemple, les obligations ou CDS).

Alors les seuils $C_i(t)$ peuvent être calculés comme suit :
\begin{equation}
C_i(t) = \Phi^{-1}(q_i(t)),
\end{equation}


Dans notre cas, la probabilité risque neutre a été calculée avec un modèle a intensité. Autrement dit nous avons considéré que le temps d'arrèt ( temps auquel le défaut interviendra suit un processus de poisson dont on a supposé l'intensité constante ). En réalité, de manière rigoureuse, l'intensité de défaut doit etre calculé avec ce que l'on appelle **le triangle de crédit**. Il s'agit d'une relation mathématique qui lie le spread de crédit de chaque actif constitutifs du portefeuilles de crédits, le taux de recouvrement et l'intensité $\lambda$ du défaut. Dans notre cas, a défaut d'avoir le spread de crédit de chaque actif présent sur le marché, nous avons choisi une valeur arbitraire de l'intensité comme input de notre modèle ( 0.2 )

Voici comment nous avons procédé dans notre modélisation : **Pricing des CDO synthétiques avec des temps de défauts**

Une simulation de Monte Carlo des temps de défaut nous permet d'évaluer tout dérivé de crédit dont le prix est une fonction de l'identité et du moment des défauts sur le portefeuille de référence. De tels produits comprennent les principaux produits de corrélation c'est-à-dire les paniers de défaut et la plupart des variations sur le CDO.
Le principal défi lors de la tarification des dérivés de crédit à l'aide de Monte Carlo est la minimisation de l'erreur standard de l'estimation du prix. Étant donné que l'erreur standard ne diminue que comme $\mathcal{O}(1/\sqrt{n})$ et que le temps de Monte Carlo évolue comme $\mathcal{O}(n\cdot P)$, nous devons nous assurer que le code s'exécute le plus rapidement possible. Cela nécessite une combinaison de conception de code soignée et de l'utilisation de méthodes de réduction de variance. Des méthodes telles que les variables de contrôle, l'échantillonnage d'importance et les générateurs de nombres pseudo-aléatoires peuvent tous être utilisés.En ce qui concerne la conception de code, nous pouvons diviser la tarification de Monte Carlo des dérivés de crédit en deux approches :

- Nous utilisons les temps de défaut pour calculer directement la valeur actualisée de chaque flux de la jambe de prime et de protection dans chaque scénario, puis nous faisons la moyenne de ces valeurs pour calculer l'espérance actualisée de chaque jambe.


- Nous utilisons les temps de défaut pour calculer la courbe de survie de la tranche, que nous fournissons ensuite à nos outils d'analyse de CDS pour calculer la valeur des différentes jambes. C'est l'approche indirecte. Cela n'est bien sûr possible que si le produit peut être évalué en utilisant simplement une courbe de survie, c'est-à-dire s'il inclut des STCDO ( Single Tranche CDO Pricing ) et des paniers de défaut à perte homogène.




Utiliser l'approche directe nécessite d'écrire un algorithme qui évalue les jambes de prime et de protection étant donné un vecteur de N temps de défaut générés aléatoirement pour chacun des P essais de Monte Carlo. Nous notons ce vecteur avec $\{ \tau_p^i \}$ et supposons que ces temps de défaut ont déjà été générés. Les étapes sont les suivantes :

- Nous définissons $p=1$.
- Nous supprimons les temps de défaut $\tau_p^i > T$ où $T$ est la maturité finale du contrat.
- Nous ordonnons les temps de défaut $\tau_p^i$ par ordre croissant en utilisant un algorithme de tri efficace tel que QuickSort, comme décrit dans Press et al. (1992).
- Nous parcourons les temps de défaut de tous les crédits qui font défaut avant le temps $T$ et calculons les flux sur les jambes de protection et de paiement. Nous connaissons l'identité des crédits en défaut, nous pouvons donc attribuer le taux de recouvrement approprié.
- Nous actualisons les flux sur la jambe principale et de protection en utilisant la courbe Libor pour donner la valeur actualisée de la jambe de protection $PV_p$ et $RPV01_p$.
- Nous définissons $p=p+1$. Si $p \leq P$, nous revenons à l'étape (2).
- Nous calculons la moyenne des valeurs actualisées de la jambe de prime et de protection:

$$ Tranche Risky PV01 = \frac{1}{P} \sum_{p = 1}^{P} RPVO1_p $$ et la moyenne de la jambe de protection est :

$$ Tranche Protection leg PV = \sum_{p = 1}^{P} Protection leg PV_p $$


L'avantage de cette approche est qu'elle est très générique et peut gérer n'importe quel dérivé de crédit dont les paiements dépendent des temps de défaut et des pertes associées à chaque crédit. Par exemple, elle peut gérer les paniers de défaut de n-ième à défaut où les pertes sont inhomogènes. Bien qu'elle fasse de nombreux appels à la fonction de facteur d'actualisation Libor, cela peut être accéléré en mettant en cache la courbe de facteur d'actualisation.


Comme décrit précédemment, nous avons suivi les memes étapes pour pricer les CDO synthétiques dans le cadre de la copule gaussienne. Pour le confort du lecteur, nous rappelons les différentes étapes pour le cas de la **copule gaussienne** :

Rappelons succinctement comment on obtient des tirages des temps d'arrèt.
On réalise d'abord un tirage $u = (u_1, \dots, u_n)$ d'une loi normale $N(\cdot; \rho)$ M-dimensionnelle à composantes centrées réduites et corrélées selon une matrice de corrélation $[\rho]$ donnée. Cette dernière est choisie en fonction de la corrélation présumée des événements de crédit. Un tel tirage peut faire appel à une décomposition de Choleski, conformément à la méthode exposée dans le cour de monte carlo. Une méthode simple pour déterminer les corrélations et effectuer les tirages est de s'appuyer sur un modèle mono-factoriel : $U_i = \rho_i L + \sqrt{1 - \rho_i^2} \varepsilon_i$, où $U_i, L$ et $\varepsilon_i$ sont des gaussiennes standardisées et $\varepsilon_i$ est indépendante de $L$ et de $\varepsilon_j$ pour $i \neq j$; $\rho_i$ est la corrélation de $U_i$ avec la conjoncture globale $L$ et $\text{corr}(U_i, U_j) = \rho_i \rho_j$. À ce tirage gaussien $(u_1, \dots, u_n)$, on fait correspondre un vecteur de M dates de défaut par la relation:
\begin{equation}
  \quad T_i^{(p)} = (\Phi^{-1}(N(u_1)), \dots, \Phi^{-1}(N(u_n)))
\end{equation}
Quand les probabilités de défaut sont déterminées à l'aide d'un modèle à intensité, l'équation précédente donne : $$T_i^{(p)} = (-\frac{1}{\lambda_i} \ln(N(u_i)), \dots, -\frac{1}{\lambda_n} \ln(N(u_n)))$$
Ce vecteur $T^{(p)}$ constitue un élément d'une simulation des M dates de défaut, présumé tiré de la loi jointe des $T_i$. On remarquera que la loi marginale de la i-ième composante est bien $\Phi^{-1}$ et que les différentes composantes sont liées par la corrélation de copule $[\rho]$.

En répétant le tirage gaussien p fois et en appliquant à chaque tirage la seconde relation des temps d'arrèts , on construit un échantillon de p éléments M-dimensionnels $(T^{(1)}, \dots, T^{(p)})$ constituant une simulation de Monte Carlo des dates de défaut.



Pour résumer, voici ce que fait notre code étape par étape :


# Classe GaussianCDOPricing

Cette classe simule le pricing d'un CDO synthétique en utilisant la copule gaussienne.

### \_\_init\_\_

Initialise les attributs de la classe :

- attachment : point d'attachement
- detachment : point de détachement
- rho : corrélation
- intensity : intensité de défaut
- T : maturité du CDO
- d : nombre d'actifs du CDO
- r : taux d'intérêt constant
- notionel : nominal
- n_samples : nombre de simulations

### stopping_times_simulation

Simule les temps d'arrêt en utilisant la copule gaussienne :

- Initialise la matrice de corrélation et génère des échantillons multivariés gaussiens.
- Utilise la méthode antithétique pour améliorer l'efficacité de la simulation.
- Corrèle les échantillons multivariés en utilisant la décomposition de Cholesky.
- Calcule les temps d'arrêt en utilisant la fonction de répartition inverse de la distribution gaussienne standard.
- Filtre les temps d'arrêt pour ne garder que ceux qui sont inférieurs à la maturité du CDO.
- Retourne un tableau numpy des temps d'arrêt filtrés.

### risk_neutral_probabilities

Calcule les probabilités neutres au risque en utilisant l'intensité de défaut.

### expected_loss

Calcule la perte attendue à un instant donné :

- Initialise la matrice de covariance bivariée et la distribution gaussienne bivariée.
- Calcule les quantiles inverses des points d'attachement et de détachement.
- Calcule la perte attendue en utilisant la fonction de répartition de la distribution bivariée et les quantiles inverses.

L'expected loss est donnée par :

$$E L_{\left(K_1, K_2\right)}(t)=\frac{\Phi_2\left(-\Phi^{-1}\left(K_1\right), C(t), \rho\right)-\Phi_2\left(-\Phi^{-1}\left(K_2\right), C(t), \rho\right)}{K_2-K_1}$$ avec $\rho$ la matrice de corrélation $$[[1, -\sqrt{1 - a^2}], \\
[-\sqrt{1 - a^2}, 1]]$$, avec $C(t)$ donnée par la formule
\begin{equation}
C(t) = \Phi^{-1}(q(t)),
\end{equation}


avec $q(t)$

\begin{equation}
q(t) = 1 - \exp(-\lambda * t)
\end{equation}

ou $\lambda$ est l'intensité de défaut. Cela est possible puisque la loi normale étant stable par convolution, les
$A_i(t)$ suivent bien une loi normale centrée et réduite. Le portefeuille étant homogène, tous les actifs ont les memes composantes.

Comme nous le voyons avec une copule gaussienne, nous voyons que l'expected loss a une expression fermée. Nous avons eu a l'implémenter directement sur Python. Nous disposions de toutes les variables qui sont des données du problèmes.

### synthetic_cdo_spread_computation_gaussian_normal

Calcule le spread du CDO synthétique en utilisant la méthode de Monte-Carlo :

- Initialise une liste pour stocker les spreads.
- Pour chaque échantillon de temps d'arrêt simulé :
  - Initialise des listes pour stocker les flux de primes et de protection.
  - Pour chaque paire de temps d'arrêt adjacents :
    - Calcule les pertes attendues aux temps d'arrêt courant et suivant en considérant le taux de recouvrement.
    - Calcule la différence de temps entre les temps d'arrêt courant et suivant.
    - Calcule le facteur d'actualisation en utilisant le taux d'intérêt constant $r$ et le temps d'arrêt courant.
    - Calcule la différence des flux de primes et de protection en utilisant les pertes attendues, la différence de temps et le facteur d'actualisation.
    - Ajoute les différences de flux de primes et de protection aux listes correspondantes.
  - Calcule la jambe de prime et la jambe de protection en sommant les flux de primes et de protection, respectivement.
  - Calcule le spread en divisant la jambe de prime par la jambe de protection, et ajoute le résultat à la liste des spreads.
- Calcule la moyenne, la variance et l'intervalle de confiance des spreads.

- Retourne un DataFrame pandas contenant les résultats.


**Taux de Recouvrement en cas de défaut**

Le taux de recouvrement est le pourcentage du nominal que l'on espère percevoir en cas de défaut. Vu que nous travaillons avec des tranches de CDO synthétiques et que $K_1$ et $K_2$ , qui sont respectivement les points d'attachements et de détachements sont **compris entre 0 et 1** dans l'article, le taux de recouvrement dénoté $R$ doit respecter certaines contraintes mathématiques majeures sous peines de ne pas pouvoir calculer les quantiles associés :

$$R > 0$$ et $$R < 1 - K_2$$ Par conséquent, $$0 < R < 1 - K_2$$

In [2]:
class GaussianCDOPricing:

    def __init__(self, attachment, detachment, rho, intensity, T, d, r, notionel, n_samples):

        self.attachment = attachment   # point d'attachement (limite inférieure de la tranche de CDO)
        self.detachment = detachment   # point de détachement (limite supérieure de la tranche de CDO)
        self.rho = rho  # corrélation entre les actifs du CDO
        self.intensity = intensity   # intensité de défaut des actifs
        self.T = T   # maturité du CDO
        self.d = d   # nombre d'actifs du CDO
        self.r = r   # taux d'intérêt constant
        self.notionel = notionel    # nominal du CDO
        self.n_samples = n_samples  # nombre de simulations pour la méthode Monte Carlo

    # Simulation des temps d'arrêt (défauts) pour les actifs du CDO
    def stopping_times_simulation(self):
        np.random.seed(42)

        mean = np.repeat(0, self.d)
        covariance_matrix = np.eye(self.d)

        half_n_samples = self.n_samples // 2
        multivariate_samples = np.random.multivariate_normal(mean, covariance_matrix, half_n_samples)
        antithetic_samples = -multivariate_samples

        multivariate_samples = np.vstack((multivariate_samples, antithetic_samples))

        covariance_matrix = np.full((self.d, self.d), self.rho) + (1 - self.rho) * np.eye(self.d)

        if np.allclose(covariance_matrix, covariance_matrix.T):
            try:
                L = np.linalg.cholesky(covariance_matrix)

            except np.linalg.LinAlgError:
                print("La matrice n'est pas sémi définie positive")

        multivariate_samples_correlated = L @ multivariate_samples.T
        multivariate_samples_correlated = multivariate_samples_correlated.T

        stopping_times = (-1 / self.intensity) * np.log(norm.cdf(multivariate_samples_correlated))

        for i in range(stopping_times.shape[0]):
            stopping_times[i] = np.sort(stopping_times[i])

        filtered_stopping_times = []

        for i in range(stopping_times.shape[0]):
            filtered_times = stopping_times[i][stopping_times[i] < self.T]
            filtered_stopping_times.append(filtered_times)

        filtered_stopping_times = np.array(filtered_stopping_times, dtype=object)

        return filtered_stopping_times

    # Calcul des probabilités neutres au risque pour un temps donné
    def risk_neutral_probabilities(self, t):

        return 1 - np.exp(-self.intensity * t)

    # Calcul de la perte attendue pour un temps donné
    def expected_loss(self, t, recovery):

        covariance_matrix = np.array([[1, -np.sqrt(1-self.rho**2)],
                                      [-np.sqrt(1-self.rho**2), 1]])

        mean = np.array([0, 0])

        bivariate_distribution = multivariate_normal(mean, covariance_matrix)

        inverse_K_1 = - norm.ppf(self.attachment/(1 - recovery))

        inverse_K_2 = - norm.ppf(self.detachment/(1 - recovery))

        x = np.array([inverse_K_1, norm.ppf(self.risk_neutral_probabilities(t))])

        y = np.array([inverse_K_2, norm.ppf(self.risk_neutral_probabilities(t))])

        # Calcul de la perte attendue en fonction de la distribution bivariée
        return (bivariate_distribution.cdf(x) -
                 bivariate_distribution.cdf(y)) / ((self.detachment - self.attachment)/(1 - recovery))

    # Calcul du spread synthétique du CDO en utilisant l'approximation gaussienne
    def synthetic_cdo_spread_computation_gaussian_normal(self, proba,recovery):

        spread_list = []

        stopping_times_sim = self.stopping_times_simulation()

        for i in range(len(stopping_times_sim)):
            premium_list = []
            protection_list = []

            stopping_times_sample = stopping_times_sim[i]

            for j in range(len(stopping_times_sample) - 1):
                current_time = stopping_times_sample[j]
                next_time = stopping_times_sample[j + 1]
                current_expected_loss = self.expected_loss(current_time,recovery)
                next_expected_loss = self.expected_loss(next_time,recovery)
                time_diff = next_time - current_time
                discount_factor = np.exp(-self.r * current_time)

                # Calcul de la différence des flux de primes entre les deux temps d'arrêt
                premium_flow_difference = (next_expected_loss - current_expected_loss) * discount_factor
                premium_list.append(premium_flow_difference)

                # Calcul de la différence des flux de protection entre les deux temps d'arrêt
                protection_flow_difference = time_diff * (1 - current_expected_loss) * discount_factor
                protection_list.append(protection_flow_difference)

            epsilon = 1e-10
            premium_leg = np.sum(premium_list)
            protection_leg = np.sum(protection_list)


            if protection_leg != 0 and not math.isnan(premium_leg) and not math.isnan(protection_leg):
                spread_list.append(premium_leg / protection_leg + epsilon)

            # Calcul de la moyenne, de la variance et de l'intervalle de confiance pour le spread synthétique
            mean = np.mean(spread_list)
            var = np.var(spread_list, ddof=1)
            alpha = 1 - proba
            quantile = norm.ppf(1 - alpha / 2)  # fonction quantile
            ci_size = quantile * np.sqrt(var / np.array(spread_list).size)

        return pd.DataFrame({'Monte-Carlo Spread': mean,
                             'Variance': var,
                             'lower_confidence': mean - ci_size,
                             'upper_confidence': mean + ci_size}, index=['Gaussienne']).T

In [4]:
gaussian = GaussianCDOPricing(attachment=0.3,detachment=0.7,rho=0.3,intensity=0.2,
                              T = 5, d= 40,r=0.02,notionel=1,n_samples=100)

Dans notre code, nous avons pris :

- attachement  = 0.3  (compris entre 0 et 1 dans l'article)

- detachement  0.7  (compris entre 0 et 1 dans l'article)

- rho = 0.3 (corrélation entre les actifs du portefeuille de référence)

- intensity = 0.2  (issu du triangle de crédit)

- T = 5  (Maturité du CDO synthétique)

- d = 40 (nombre d'actifs du portefeuille de référence)

- notionel = 1 (Le montant du nominal est l'unité)

- n_samples = 100 (nombre de simulations égales a 100)

- r = 0.02

- taux de recouvrement = 0.2

Ci-joint le résultat de la simulation de la copule gaussienne. Nous remarquons qu'avec la réduction de la variance, notre variance de simulation est très faible. Ce qui est une très bonne chose.

In [5]:
%%time
df_normal = gaussian.synthetic_cdo_spread_computation_gaussian_normal(0.95,recovery= 0.2)

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:4008: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:210: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


CPU times: user 18.3 s, sys: 189 ms, total: 18.5 s
Wall time: 19.9 s


In [6]:
df_normal

,Gaussienne
Monte-Carlo Spread,0.125615
Variance,0.000610
lower_confidence,0.120774
upper_confidence,0.130456


**Copule de Student t a un facteur et Approximation LHP**

La copule de Student a été une des copules les plus introduites comme modèle de pricing de CDO synthétiques. La raison profonde est un incovénient majeur de la copule gaussienne. La copule de Student-t est l'une des copules les plus utilisées. Elle est basée sur la version multivariée de la distribution de Student-t. Comme la copule gaussienne, la copule de Student-t fait également partie de la famille des copules elliptiques. Cependant, la distribution de Student-t peut présenter des queues plus épaisses que la distribution gaussienne et est donc mieux adaptée pour modéliser la dépendance des queues. En effet, si nous calculons les corrélations implicites des prix du marché des tranches d'un même CDO en utilisant l'approche LHP, nous n'obtenons pas la même corrélation sur l'ensemble de la structure, mais plutôt, nous observons une distorsion de corrélation (correlation skew). La principale explication de ce phénomène est le manque de dépendance dans les queues de la copule gaussienne. En effet, la copule gaussienne a des queues fines, ce qui en soit, sur les marchés financiers, ne reproduit pas les quantités indispensables pour le trading ou le hedging.

Comme la copule gaussienne, la copule de Student-$t$ fait également partie de la famille des copules elliptiques. Cependant, la distribution de Student-$t$ peut présenter des queues plus épaisses que la distribution gaussienne et est donc mieux adaptée pour modéliser la dépendance des queues.

On dit qu'une variable aléatoire $X$ suit une distribution de Student-$t$ si elle est générée selon la formule suivante :
$$
X=\frac{Z}{\sqrt{\xi_v / v}}
$$
où $Z \sim N(0,1)$ et $\xi_v$ est une variable aléatoire indépendante distribuée selon une loi du khi-deux avec $v$ degrés de liberté. La fonction de densité de probabilité de la distribution de Student-$t$ est donnée par :
$$
f_v(t)=\frac{\Gamma((v+1) / 2)}{\sqrt{v \pi} \Gamma(v / 2)}\left(1+\frac{t^2}{v}\right)^{-(v+1) / 2}
$$
où $\Gamma(x)$ est la fonction gamma. La distribution de Student-$t$ a les propriétés suivantes :

- Elle est symétrique autour de $t=0$ et a donc une moyenne nulle.
- Elle a une variance égale à $\frac{v}{v-2}$ où le paramètre $v$ est connu sous le nom de degrés de liberté. La variance est donc définie seulement si $v>2$.
- À la limite $v \rightarrow \infty$, la distribution converge vers une distribution gaussienne. Ceci est illustré dans la Figure qui représente la fonction de densité de probabilité pour différentes valeurs de $v$.


Le modèle LHP de Student est basé sur une copule de Student. Supposons que le portefeuille d'actifs de référence se compose de $m$ instruments financiers et que le rendement de l'actif jusqu'au temps $t$ du $i$-ème émetteur du portefeuille, $A_i(t)$, ait la forme :
\begin{equation}
A_i(t) = a_i M(t) + \sqrt{1 - a_i^2} X_i(t), \qquad
\end{equation}
où $M(t)$, $X_i(t)$, $i = 1, \dots, m$, sont des variables aléatoires indépendantes suivant une distribution de Student d'un certain dégré de liberté. Alors, conditionnellement au facteur de marché commun $M(t)$, les rendements des actifs des différents émetteurs sont indépendants.


Malheureusement, contrairement a la copule gaussienne, la somme de deux variables aléatoires de Student ne donne pas nécessairement une loi de Student. Cela pose un gros problème lors du calcul des quantiles puisque l'on ne dispose pas de la fonction de répartition $A_i(t)$. Une pratique commune en pratique consiste a choisir une loi de probabilité qui se rapproche de plus des $A_i(t)$.


**Dans notre cas, nous avons décidé de choisir une loi qui se rapproche le plus des $A_i(t)$. Notre choix s'est fait de telle sorte a considérer une loi de probabilité dont les paramètres dépendent fortement des dégrés de liberté de la loi de Student. Nous n'avons pas voulu choisir une loi de probabilité avec des paramètres moyenne et variance a estimer. En se placant toujours dans un cadre simplifié, nous allons considérer une loi de student. Autrement dit, nous allons considérer que les $A_i(t)$ suivent une loi de student.**

Dans ce cas la, l'expected loss n'est plus une formule fermée comme dans le cadre gaussien et se doit d'etre approximé numériquement.


Supposons qu'une fonction de distribution de pertes de portefeuille continue $F(t, x)$ soit connue. Alors, la perte attendue en pourcentage de la tranche CDO $\left(K_{1}, K_{2}\right)$ peut être calculée comme suit :

$$
E L_{\left(K_{1}, K_{2}\right)}(t)=\frac{1}{K_{2}-K_{1}} \int_{K_{1}}^{1}\left(\min \left(x, K_{2}\right)-K_{1}\right) d F(t, x)
$$

La perte attendue de la tranche dans l'équation peut également être écrite comme suit :

$$
E L_{\left(K_{1}, K_{2}\right)}(t)=\frac{1}{K_{2}-K_{1}}\left(\int_{K_{1}}^{1}\left(x-K_{1}\right) d F(t, x)-\int_{K_{2}}^{1}\left(x-K_{2}\right) d F(t, x)\right) .
$$

Une extension naturelle de l'approche LHP consiste à utiliser une hypothèse de distribution qui produit une queue lourde. Le modèle à un facteur double $t$ proposé par Hull et White [2004] suppose des distributions de Student t pour le facteur de marché commun $M(t)$ ainsi que pour les facteurs individuels $X_{i}(t)$. Alors, la distribution de perte $F_{\infty}(t, x)$ dans l'équation devient :

$$
F_{\infty}(t, x)=T\left(\frac{\sqrt{1-a^{2}} T^{-1}(x)-C(t)}{a}\right)
$$

où $T$ désigne la fonction de distribution de Student t.

In [7]:
class Student_t_copula:

    def __init__(self, attachment, detachment, rho, intensity, T, d, r, notionel, n_samples, nu):
        self.attachment = attachment
        self.detachment = detachment
        self.rho = rho
        self.intensity = intensity
        self.T = T
        self.d = d
        self.r = r
        self.notionel = notionel
        self.n_samples = n_samples
        self.nu = nu


    def stopping_times_simulation(self):
        np.random.seed(42)

        mean = np.repeat(0, self.d)
        covariance_matrix = np.eye(self.d)

        half_n_samples = self.n_samples // 2
        multivariate_samples = np.random.multivariate_normal(mean, covariance_matrix, half_n_samples)
        antithetic_samples = - multivariate_samples

        multivariate_samples = np.vstack((multivariate_samples, antithetic_samples))

        covariance_matrix = np.full((self.d, self.d), self.rho) + (1 - self.rho) * np.eye(self.d)

        if np.allclose(covariance_matrix, covariance_matrix.T):
            try:
                L = np.linalg.cholesky(covariance_matrix)

            except np.linalg.LinAlgError:
                print("La matrice n'est pas sémi définie positive")

        multivariate_samples_correlated = L @ multivariate_samples.T
        multivariate_samples_correlated = multivariate_samples_correlated.T

        stopping_times = (-1 / self.intensity) * np.log(norm.cdf(multivariate_samples_correlated))

        for i in range(stopping_times.shape[0]):
            stopping_times[i] = np.sort(stopping_times[i])

        filtered_stopping_times = []

        for i in range(stopping_times.shape[0]):
            filtered_times = stopping_times[i][stopping_times[i] < self.T]
            filtered_stopping_times.append(filtered_times)

        filtered_stopping_times = np.array(filtered_stopping_times, dtype=object)

        return filtered_stopping_times


    def risk_neutral_probabilities(self,t):

        return 1 - np.exp(- self.intensity * t)




    def student_copula_percentile(self,time,index):

        proba = self.risk_neutral_probabilities(time)

        quantile = t.ppf(proba,self.nu)

        return quantile


    def expected_loss(self,time,index,recovery=0):

        def f(x):

            return (min(x,(self.detachment/(1 - recovery))) - (self.attachment/(1 - recovery)))

        def g(x):

            c_t = self.student_copula_percentile(time,index)

            return (np.sqrt(1-self.rho**2)/self.rho) * (1/t.pdf(t.ppf(x,self.nu),self.nu)) * t.pdf((np.sqrt(1-self.rho**2) * t.ppf(x,self.nu) - c_t)/self.rho,self.nu)

        def integrand(x):

            return (f(x) * g(x))/((self.detachment - self.attachment)/(1 - recovery))


        a = (self.attachment/(1 - recovery))
        b = 1

        return quad(integrand,a,b)[0]


    def synthetic_cdo_spread_computation_student(self,proba,recovery=0):

         spread_list = []

         stopping_times_sim = self.stopping_times_simulation()

         for i in range(len(stopping_times_sim)):
            premium_list = []
            protection_list = []

            stopping_times_sample = stopping_times_sim[i]

            for j in range(len(stopping_times_sample) - 1):
                current_time = stopping_times_sample[j]
                next_time = stopping_times_sample[j + 1]
                current_expected_loss = self.expected_loss(current_time,i,recovery)
                next_expected_loss = self.expected_loss(next_time,i,recovery)
                time_diff = next_time - current_time
                discount_factor = np.exp(-self.r * current_time)

                premium_flow_difference = (next_expected_loss - current_expected_loss) * discount_factor
                premium_list.append(premium_flow_difference)

                protection_flow_difference = time_diff * (1 - current_expected_loss) * discount_factor
                protection_list.append(protection_flow_difference)

            epsilon = 1e-10
            premium_leg = np.sum(premium_list)
            protection_leg = np.sum(protection_list)

            if protection_leg != 0 and not math.isnan(premium_leg) and not math.isnan(protection_leg):
                spread_list.append(premium_leg / protection_leg + epsilon)


            mean = np.mean(spread_list)
            var = np.var(spread_list, ddof=1)
            alpha = 1 - proba
            quantile = norm.ppf(1 - alpha/2)  # fonction quantile
            ci_size = quantile * np.sqrt(var / np.array(spread_list).size)

         return pd.DataFrame({'Monte-Carlo Spread':mean,
                             'Variance':var,
                             'lower_confidence':mean - ci_size,
                             'upper_confidence':mean + ci_size},index=['Student']).T


In [8]:
nig = Student_t_copula(attachment=0.3,detachment=0.7,rho=0.3, intensity=0.2, T=5,
                                     d = 40, r=0.02, notionel=1, n_samples=5, nu=10)

In [9]:
%%time
df_nig_student = nig.synthetic_cdo_spread_computation_student(0.95,recovery = 0.2)

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:4008: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:210: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


CPU times: user 1min 43s, sys: 5.98 s, total: 1min 49s
Wall time: 1min 46s


In [10]:
df_nig_student

,Student
Monte-Carlo Spread,0.126597
Variance,0.000061
lower_confidence,0.118941
upper_confidence,0.134254


Nous obtenons des résultats sensiblement égaux a la loi normale. La simualtion est assez lente. Dans notre cas, nous avons fait 5 simulations mais nous remarquons déjà que les résultats semblent converger **rapidement** vers les résultats de la loi copule gaussienne.

In [12]:
pd.concat([df_normal,df_nig_student],axis=1)

,Gaussienne,Student
Monte-Carlo Spread,0.125615,0.126597
Variance,0.000610,0.000061
lower_confidence,0.120774,0.118941
upper_confidence,0.130456,0.134254
